In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!pip install flask

In [13]:
!pip install flask-ngrok

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from PIL import Image,ImageFilter
import cv2
import keras
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda, Reshape
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.optimizers import Adam

In [97]:
def ConvLayers(layers, model, filters):
  for i in range(layers):
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(filters, 3,3,activation='relu'))
  model.add(MaxPooling2D((2,2),strides=(1,1),padding="same"))
def FCLayers(model):
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))
def VGG16_arc():
  model=Sequential()
  ConvLayers(2,model,64)
  ConvLayers(2,model,128)
  ConvLayers(3,model,256)
  ConvLayers(3,model,512)
  ConvLayers(3,model,512)
  model.add(Flatten())
  FCLayers(model)
  FCLayers(model)
  model.add(Dense(37,activation='sigmoid'))
  #model.add(Dense(3,activation='softmax'))
  return model

In [98]:
optimizer = Adam(lr=0.001, decay=5e-4)
model = VGG16_arc()
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [99]:
model.build(input_shape=(None,224,224,3))

In [100]:
model.load_weights('/content/drive/MyDrive/flask_test/models/thirdrunweights.hdf5')

In [180]:
def preprocessing(imagepath):
  img=Image.open(imagepath)
  resized=img.resize((224,224))
  medfil=resized.filter(ImageFilter.MedianFilter(size = 3)) 
  open_cv_image = np.array(medfil) 
  # Convert RGB to BGR 
  open_cv_image = open_cv_image[:, :, ::-1].copy() 
  ycb=cv2.cvtColor(open_cv_image,cv2.COLOR_BGR2LAB) # Converting RGB to LAB colorscale
  imgs=cv2.split(ycb)  # Splitting Channels of LAB Colorscale.
  clahe=cv2.createCLAHE(clipLimit=2.0,tileGridSize=(3,3)) # Creating CLAHE(Clip limit:sets threshhold for contrast limiting.)
  # tilegridesize:used for subimages for applying clahe.
  imgs[0]= clahe.apply(imgs[0]) # Applying Clahe on luminescence channel
  labimg=cv2.merge(imgs)   # Merging channels of LAB.
  labimg=cv2.cvtColor(labimg,cv2.COLOR_LAB2BGR) # Converting back LAB to RGB color scale in BGR format
  RGBimage = labimg[...,::-1] #BGR to RGB
  return RGBimage

In [181]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

def prediction(image):
  img_array = img_to_array(image)
  img_batch = np.expand_dims(img_array, axis=0)
  predictarray=model.predict(img_batch)
  return predictarray

predictarray=prediction(image)

In [182]:
classes = ['Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1','Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3','Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3','Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5', 'Class8.6', 'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1', 'Class10.2', 'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 'Class11.4', 'Class11.5', 'Class11.6'
]

In [189]:
features=[]
def class7(predictdf):
  global features
  if ((predictdf['Class7.1']>predictdf['Class7.2']) & (predictdf['Class7.1']>predictdf['Class7.3'])).any():
    print('The Galaxy Is Completely Round')
    features.append('The Galaxy Is Completely Round')
    class6(predictdf)
  elif ((predictdf['Class7.2']>predictdf['Class7.1']) & (predictdf['Class7.2']>predictdf['Class7.3'])).any():
    print('The Galaxy Is Neither Completely Round Nor Cigar Shaped (In Between)')
    features.append('The Galaxy Is Neither Completely Round Nor Cigar Shaped (In Between)')
    class6(predictdf)
  elif ((predictdf['Class7.3']>predictdf['Class7.1']) & (predictdf['Class7.3']>predictdf['Class7.2'])).any():
    print('The Galaxy Is Cigar Shaped')
    features.append('The Galaxy Is Cigar Shaped')
    class6(predictdf)

def class6(predictdf):
  global features
  if (predictdf['Class6.1']>predictdf['Class6.2']).any():
    class8(predictdf)
  elif (predictdf['Class6.2']>predictdf['Class6.1']).any():
    print('The Galaxy Has No Odd Feature')
    features.append('The Galaxy Has No Odd Feature')

def class8(predictdf):
  global features
  if ((predictdf['Class8.1']>predictdf['Class8.2']) & (predictdf['Class8.1']>predictdf['Class8.3']) & (predictdf['Class8.1']>predictdf['Class8.4']) & (predictdf['Class8.1']>predictdf['Class8.5']) & (predictdf['Class8.1']>predictdf['Class8.6']) & (predictdf['Class8.1']>predictdf['Class8.7'])).any():
    print('Odd Feature - Ring')
    features.append('Odd Feature - Ring')
  elif ((predictdf['Class8.2']>predictdf['Class8.1']) & (predictdf['Class8.2']>predictdf['Class8.3']) & (predictdf['Class8.2']>predictdf['Class8.4']) & (predictdf['Class8.2']>predictdf['Class8.5']) & (predictdf['Class8.2']>predictdf['Class8.6']) & (predictdf['Class8.2']>predictdf['Class8.7'])).any():
    print('Odd Feature - Lens Or Arc')
    features.append('Odd Feature - Lens Or Arc')
  elif ((predictdf['Class8.3']>predictdf['Class8.1']) & (predictdf['Class8.3']>predictdf['Class8.2']) & (predictdf['Class8.3']>predictdf['Class8.4']) & (predictdf['Class8.3']>predictdf['Class8.5']) & (predictdf['Class8.3']>predictdf['Class8.6']) & (predictdf['Class8.3']>predictdf['Class8.7'])).any():
    print('Odd Feature - Disturbed')
    features.append('Odd Feature - Disturbed')
  elif ((predictdf['Class8.4']>predictdf['Class8.1']) & (predictdf['Class8.4']>predictdf['Class8.2']) & (predictdf['Class8.4']>predictdf['Class8.3']) & (predictdf['Class8.4']>predictdf['Class8.5']) & (predictdf['Class8.4']>predictdf['Class8.6']) & (predictdf['Class8.4']>predictdf['Class8.7'])).any():
    print('Odd Feature - Irregular')
    features.append('Odd Feature - Irregular')
  elif ((predictdf['Class8.5']>predictdf['Class8.1']) & (predictdf['Class8.5']>predictdf['Class8.2']) & (predictdf['Class8.5']>predictdf['Class8.3']) & (predictdf['Class8.5']>predictdf['Class8.4']) & (predictdf['Class8.5']>predictdf['Class8.6']) & (predictdf['Class8.5']>predictdf['Class8.7'])).any():
    print('Odd Feature - Other')
    features.append('Odd Feature - Other')
  elif ((predictdf['Class8.6']>predictdf['Class8.1']) & (predictdf['Class8.6']>predictdf['Class8.2']) & (predictdf['Class8.6']>predictdf['Class8.3']) & (predictdf['Class8.6']>predictdf['Class8.4']) & (predictdf['Class8.6']>predictdf['Class8.5']) & (predictdf['Class8.6']>predictdf['Class8.7'])).any():
    print('Odd Feature - Merger')
    features.append('Odd Feature - Merger')
  elif ((predictdf['Class8.7']>predictdf['Class8.1']) & (predictdf['Class8.7']>predictdf['Class8.2']) & (predictdf['Class8.7']>predictdf['Class8.3']) & (predictdf['Class8.7']>predictdf['Class8.4']) & (predictdf['Class8.7']>predictdf['Class8.5']) & (predictdf['Class8.7']>predictdf['Class8.6'])).any():
    print('Odd Feature - Dust Lane')
    features.append('Odd Feature - Dust Lane')

def class2(predictdf):
  global features
  if (predictdf['Class2.1']>predictdf['Class2.2']).any():
    print('The Disk Can Be Viewed Edge-on')
    features.append('The Disk Can Be Viewed Edge-on')
    class9(predictdf)
  elif (predictdf['Class2.2']>predictdf['Class2.1']).any():
    print('The Disk Cannot Be Viewed Edge-on')
    features.append('The Disk Cannot Be Viewed Edge-on')
    class3(predictdf)

def class9(predictdf):
  global features
  if ((predictdf['Class9.1']>predictdf['Class9.2']) & (predictdf['Class9.1']>predictdf['Class9.3'])).any():
    print('The Galaxy Has A Rounded Bulge At Its Centre')
    features.append('The Galaxy Has A Rounded Bulge At Its Centre')
    class6(predictdf)
  elif ((predictdf['Class9.2']>predictdf['Class9.1']) & (predictdf['Class9.2']>predictdf['Class9.3'])).any():
    print('The Galaxy Has A Boxy Bulge At Its Centre')
    features.append('The Galaxy Has A Boxy Bulge At Its Centre')
    class6(predictdf)
  elif ((predictdf['Class9.3']>predictdf['Class9.1']) & (predictdf['Class9.3']>predictdf['Class9.2'])).any():
    print('The Galaxy Has No Bulge At Its Centre ')
    features.append('The Galaxy Has No Bulge At Its Centre ')
    class6(predictdf)
  
def class3(predictdf):
  global features
  if (predictdf['Class3.1']>predictdf['Class3.2']).any():
    print('There Is A Sign Of A Bar Feature Through The Centre Of The Galaxy')
    features.append('There Is A Sign Of A Bar Feature Through The Centre Of The Galaxy')
    class4(predictdf)
  elif (predictdf['Class3.2']>predictdf['Class3.1']).any():
    print('There Is No Sign Of A Bar Feature Through The Centre Of The Galaxy')
    features.append('There Is No Sign Of A Bar Feature Through The Centre Of The Galaxy')
    class4(predictdf)
  
def class4(predictdf):
  global features
  if (predictdf['Class4.1']>predictdf['Class4.2']).any():
    print('There Is A Sign Of A Spiral Arm Pattern')
    features.append('There Is A Sign Of A Spiral Arm Pattern')
    class10(predictdf)
  elif (predictdf['Class4.2']>predictdf['Class4.1']).any():
    print('There Is No Sign Of A Spiral Arm Pattern')
    features.append('There Is No Sign Of A Spiral Arm Pattern')
    class5(predictdf)

def class10(predictdf):
  global features
  if ((predictdf['Class10.1']>predictdf['Class10.2']) & (predictdf['Class10.1']>predictdf['Class10.3'])).any():
    print('The Spiral Arms Appear To Be Tightly Wound')
    features.append('The Spiral Arms Appear To Be Tightly Wound')
    class11(predictdf)
  elif ((predictdf['Class10.2']>predictdf['Class10.1']) & (predictdf['Class10.2']>predictdf['Class10.3'])).any():
    print('The Spiral Arms Appear To Be Neither Tightly Nor Loosely Wound (Medium)')
    features.append('The Spiral Arms Appear To Be Neither Tightly Nor Loosely Wound (Medium)')
    class11(predictdf)
  elif ((predictdf['Class10.3']>predictdf['Class10.1']) & (predictdf['Class10.3']>predictdf['Class10.2'])).any():
    print('The Spiral Arms Appear To Be Loosely Wound')
    features.append('The Spiral Arms Appear To Be Loosely Wound')
    class11(predictdf)

def class5(predictdf):
  global features
  if ((predictdf['Class5.1']>predictdf['Class5.2']) & (predictdf['Class5.1']>predictdf['Class5.3']) & (predictdf['Class5.1']>predictdf['Class5.4'])).any():
    print('The Galaxy Has No Central Bulge')
    features.append('The Galaxy Has No Central Bulge')
    class6(predictdf)
  elif ((predictdf['Class5.2']>predictdf['Class5.1']) & (predictdf['Class5.2']>predictdf['Class5.3']) & (predictdf['Class5.2']>predictdf['Class5.4'])).any():
    print('The Galaxy Has A Central Bulge That Is Just Noticeable')
    features.append('The Galaxy Has A Central Bulge That Is Just Noticeable')
    class6(predictdf)
  elif ((predictdf['Class5.3']>predictdf['Class5.1']) & (predictdf['Class5.3']>predictdf['Class5.2']) & (predictdf['Class5.3']>predictdf['Class5.4'])).any():
    print('The Galaxy Has An Obvious Central Bulge')
    features.append('The Galaxy Has An Obvious Central Bulge')
    class6(predictdf)
  elif ((predictdf['Class5.4']>predictdf['Class5.1']) & (predictdf['Class5.4']>predictdf['Class5.2']) & (predictdf['Class5.4']>predictdf['Class5.3'])).any():
    print('The Galaxy Has A Dominant Central Bulge')
    features.append('The Galaxy Has A Dominant Central Bulge')
    class6(predictdf)

def class11(predictdf):
  global features
  if ((predictdf['Class11.1']>predictdf['Class11.2']) & (predictdf['Class11.1']>predictdf['Class11.3']) & (predictdf['Class11.1']>predictdf['Class11.4']) & (predictdf['Class11.1']>predictdf['Class11.5']) & (predictdf['Class11.1']>predictdf['Class11.6'])).any():
    print('The Galaxy Has 1 Spiral Arm')
    features.append('The Galaxy Has 1 Spiral Arm')
  elif ((predictdf['Class11.2']>predictdf['Class11.1']) & (predictdf['Class11.2']>predictdf['Class11.3']) & (predictdf['Class11.2']>predictdf['Class11.4']) & (predictdf['Class11.2']>predictdf['Class11.5']) & (predictdf['Class11.2']>predictdf['Class11.6'])).any():
    print('The Galaxy Has 2 Spiral Arms')
    features.append('The Galaxy Has 2 Spiral Arms')
  elif ((predictdf['Class11.3']>predictdf['Class11.1']) & (predictdf['Class11.3']>predictdf['Class11.2']) & (predictdf['Class11.3']>predictdf['Class11.4']) & (predictdf['Class11.3']>predictdf['Class11.5']) & (predictdf['Class11.3']>predictdf['Class11.6'])).any():
    print('The Galaxy Has 3 Spiral Arms')
    features.append('The Galaxy Has 3 Spiral Arms')
  elif ((predictdf['Class11.4']>predictdf['Class11.1']) & (predictdf['Class11.4']>predictdf['Class11.2']) & (predictdf['Class11.4']>predictdf['Class11.3']) & (predictdf['Class11.4']>predictdf['Class11.5']) & (predictdf['Class11.4']>predictdf['Class11.6'])).any():
    print('The Galaxy Has 4 Spiral Arms')
    features.append('The Galaxy Has 4 Spiral Arms')
  elif ((predictdf['Class11.5']>predictdf['Class11.1']) & (predictdf['Class11.5']>predictdf['Class11.2']) & (predictdf['Class11.5']>predictdf['Class11.3']) & (predictdf['Class11.5']>predictdf['Class11.4']) & (predictdf['Class11.5']>predictdf['Class11.6'])).any():
    print('The Galaxy Has More Than 4 Spiral Arms')
    features.append('The Galaxy Has More Than 4 Spiral Arms')
  elif ((predictdf['Class11.6']>predictdf['Class11.1']) & (predictdf['Class11.6']>predictdf['Class11.2']) & (predictdf['Class11.6']>predictdf['Class11.3']) & (predictdf['Class11.6']>predictdf['Class11.4']) & (predictdf['Class11.6']>predictdf['Class11.5'])).any():
    print('The Number Of Spiral Arms That The Galaxy Has Is Indeterminable')
    features.append('The Number Of Spiral Arms That The Galaxy Has Is Indeterminable')
  
def getfeatures(predictdf):
  global features
  features=[]
  print("The predicted features of the image are - \n")
  if ((predictdf['Class1.1']>predictdf['Class1.2']) & (predictdf['Class1.1']>predictdf['Class1.3'])).any():
    print('The Galaxy Is Smooth')
    features.append('The Galaxy Is Smooth')
    class7(predictdf)
  elif ((predictdf['Class1.2']>predictdf['Class1.1']) & (predictdf['Class1.2']>predictdf['Class1.3'])).any():
    print('The Galaxy Has Features/Disk')
    features.append('The Galaxy Has Features/Disk')
    class2(predictdf)
  elif ((predictdf['Class1.3']>predictdf['Class1.1']) & (predictdf['Class1.3']>predictdf['Class1.2'])).any():
    print('The Image Is Of A Star/Artifact')
    features.append('The Image Is Of A Star/Artifact')
  return features

In [190]:
from flask import Flask,url_for,render_template,request,redirect
from PIL import Image,ImageFilter
from flask_ngrok import run_with_ngrok
import io
import base64
import os
arr=[]
#UPLOAD_FOLDER = '/content/drive/MyDrive/flask_test/static/uploads'

app = Flask(__name__, template_folder='/content/drive/MyDrive/flask_test/templates/')
#app.secret_key = "secret key"
#app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
run_with_ngrok(app)


'''ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS'''

"ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])\n\ndef allowed_file(filename):\n\treturn '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS"

tried to store images using app.config(UPLOAD_FOLDER) didnt work :/ , images are being stored but not getting displayed.

In [191]:
@app.route('/')
def upload_image():
  return render_template('index.html')

In [192]:

@app.route("/upload",methods=['GET','POST'])
def upload():                                        
        if request.method == 'POST':
                f = request.files['file']
                #print(f.filename)
                #f.save(secure_filename(f.filename))
                #filename = secure_filename(f.filename)
                img=f.read()
                p=Image.open(f)
                pred=prediction(p)
                predictdf=pd.DataFrame(data= pred, columns=classes) 
                global features
                features=[]
                features=getfeatures(predictdf)
                x=Image.open(f)
                data=io.BytesIO()
                x.save(data, "JPEG")
                encoded=base64.b64encode(data.getvalue())
                return render_template('display.html', value=encoded.decode('utf-8'), length=len(features), features=features)

In [193]:
features

[]

In [194]:
app.run()



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cf31418aad9d.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Jun/2021 04:36:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2021 04:36:27] "GET /css/styles.css?v=1.0 HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2021 04:36:27] "GET /js/scripts.js HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2021 04:36:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2021 04:36:39] "POST /upload HTTP/1.1" 200 -


The predicted features of the image are - 

The Galaxy Has Features/Disk
The Disk Cannot Be Viewed Edge-on
There Is No Sign Of A Bar Feature Through The Centre Of The Galaxy
There Is No Sign Of A Spiral Arm Pattern
The Galaxy Has No Central Bulge
The Galaxy Has No Odd Feature


[2021-06-10 04:36:41,489] ERROR in app: Exception on /upload [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2098, in make_response
    "The view function did not return a valid response. The"
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [10/Jun/2021 04:36:41] "GET /upload HTTP/1.1" 500 -


In [135]:
image=load_img('/content/drive/MyDrive/flask_test/static/uploads/bg.jpg')
image=img_to_array(image)
predictarray=prediction(image)
predictdf = pd.DataFrame(data = predictarray,columns = classes)
predictdf

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,Class5.2,Class5.3,Class5.4,Class6.1,Class6.2,Class7.1,Class7.2,Class7.3,Class8.1,Class8.2,Class8.3,Class8.4,Class8.5,Class8.6,Class8.7,Class9.1,Class9.2,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,0.742251,0.191138,0.069667,0.010441,0.177414,0.018113,0.157539,0.030714,0.147438,0.010131,0.041932,0.090104,0.031597,0.240104,0.759837,0.401176,0.338155,0.003645,0.008384,0.02544,0.024866,0.01859,0.107872,0.052407,0.001826,0.008279,0.001131,0.001339,0.013592,0.010536,0.006058,0.003647,0.006172,0.000362,0.000342,0.00133,0.016744


In [136]:
features=getfeatures(predictdf)

The predicted features of the image are - 

The Galaxy Is Smooth
The Galaxy Is Completely Round
The Galaxy Has No Odd Feature


In [137]:
print(type(features))

<class 'list'>


In [138]:
features

['The Galaxy Is Smooth',
 'The Galaxy Is Completely Round',
 'The Galaxy Has No Odd Feature']